# 2D Lidar Scans to 3D Point Cloud
## Perpendicular mounting, horizontal sweep

In [1]:
import numpy as np
import open3d as o3d
import matplotlib
from matplotlib import pyplot as plt

In [2]:
mock_data = np.random.rand(120, 360) * 0.2 + 6.0 # mock data, each row is a full lidar scan from a servo position
# col represents angle from lidar
# values are ranges in meters

In [3]:
ranges = mock_data[:, 0:181]
ranges.shape

(120, 181)

In [4]:
# theta angles (lidar angles) in radians
thetas = np.radians(np.arange(181, dtype=float))
thetas[0], thetas[90], thetas[180]

(np.float64(0.0),
 np.float64(1.5707963267948966),
 np.float64(3.141592653589793))

In [5]:
# servo angles in radians
sweep = mock_data.shape[0]
min_angle = (180 - mock_data.shape[0] ) / 2
max_angle = min_angle + sweep
rhos = np.radians(np.arange(min_angle, max_angle))
rhos

array([0.52359878, 0.54105207, 0.55850536, 0.57595865, 0.59341195,
       0.61086524, 0.62831853, 0.64577182, 0.66322512, 0.68067841,
       0.6981317 , 0.71558499, 0.73303829, 0.75049158, 0.76794487,
       0.78539816, 0.80285146, 0.82030475, 0.83775804, 0.85521133,
       0.87266463, 0.89011792, 0.90757121, 0.9250245 , 0.9424778 ,
       0.95993109, 0.97738438, 0.99483767, 1.01229097, 1.02974426,
       1.04719755, 1.06465084, 1.08210414, 1.09955743, 1.11701072,
       1.13446401, 1.15191731, 1.1693706 , 1.18682389, 1.20427718,
       1.22173048, 1.23918377, 1.25663706, 1.27409035, 1.29154365,
       1.30899694, 1.32645023, 1.34390352, 1.36135682, 1.37881011,
       1.3962634 , 1.41371669, 1.43116999, 1.44862328, 1.46607657,
       1.48352986, 1.50098316, 1.51843645, 1.53588974, 1.55334303,
       1.57079633, 1.58824962, 1.60570291, 1.6231562 , 1.6406095 ,
       1.65806279, 1.67551608, 1.69296937, 1.71042267, 1.72787596,
       1.74532925, 1.76278254, 1.78023584, 1.79768913, 1.81514

In [6]:
# horizontal component of range (broadcast multiplication)
ranges_xy = ranges * np.sin(thetas)

In [7]:
# reshape rhos to vertical
rhos = rhos.reshape(-1, 1)
rhos

array([[0.52359878],
       [0.54105207],
       [0.55850536],
       [0.57595865],
       [0.59341195],
       [0.61086524],
       [0.62831853],
       [0.64577182],
       [0.66322512],
       [0.68067841],
       [0.6981317 ],
       [0.71558499],
       [0.73303829],
       [0.75049158],
       [0.76794487],
       [0.78539816],
       [0.80285146],
       [0.82030475],
       [0.83775804],
       [0.85521133],
       [0.87266463],
       [0.89011792],
       [0.90757121],
       [0.9250245 ],
       [0.9424778 ],
       [0.95993109],
       [0.97738438],
       [0.99483767],
       [1.01229097],
       [1.02974426],
       [1.04719755],
       [1.06465084],
       [1.08210414],
       [1.09955743],
       [1.11701072],
       [1.13446401],
       [1.15191731],
       [1.1693706 ],
       [1.18682389],
       [1.20427718],
       [1.22173048],
       [1.23918377],
       [1.25663706],
       [1.27409035],
       [1.29154365],
       [1.30899694],
       [1.32645023],
       [1.343

In [8]:
ranges_z = ranges * np.cos(thetas)
ranges_z

array([[ 6.18617131,  6.06318505,  6.02586232, ..., -6.1737219 ,
        -6.05200592, -6.00837963],
       [ 6.01783491,  6.10934083,  6.16628264, ..., -6.07959373,
        -6.19527194, -6.12097358],
       [ 6.11728161,  6.0102246 ,  6.03895264, ..., -6.08651644,
        -6.17801933, -6.1205588 ],
       ...,
       [ 6.18778551,  6.15075521,  6.14285554, ..., -6.06800439,
        -6.10230209, -6.15224068],
       [ 6.07689258,  6.09297951,  6.04141567, ..., -6.0685144 ,
        -6.14850412, -6.1797907 ],
       [ 6.18035726,  6.15238994,  6.19582947, ..., -6.02341813,
        -6.19356455, -6.02598398]])

In [9]:
x = ranges_xy * np.cos(rhos)
y = ranges_xy * np.sin(rhos)

x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)
z = ranges_z.flatten().reshape(-1, 1)

In [10]:
xyz = np.hstack((x, y, z))
xyz.shape

(21720, 3)

In [11]:
# Create point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
o3d.io.write_point_cloud("test.ply", pcd)

True

In [ ]:
# Visualize
# mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
pcd.estimate_normals()
pcd_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd)
pcd_mesh.paint_uniform_color([1, 0, 0])
o3d.web_visualizer.draw_geometries([pcd_mesh])


: 